In [1]:
import os
import re
import sys

import numpy as np
import matplotlib as plt

from dataReader import getLabels, getOntologies
from baselines import normalizedLevenshtein

In [5]:
true_labels,true_paths,true_matches,_,true_sources,true_ontologies,_ = getLabels(path='Copy of mapping.xlsx')

In [6]:
def labelDict(data):
    dictionary = {}
    for key,val1,val2 in data:
        key = key.strip().lower()
        if key in dictionary:
            if (val1,val2) not in dictionary[key]:
                dictionary[key] += [(val1.lower(), val2.lower() + '.txt')]
        else:
            dictionary[key] = [(val1.lower(), val2.lower() + '.txt')]
    return dictionary

In [7]:
dictionary = labelDict(zip(true_labels,true_matches,true_ontologies))
dictionary2 = labelDict(zip(true_labels,true_paths,true_ontologies))
#dictionary = labelDict(zip(true_labels,true_sources,true_ontologies))

In [8]:
f = open('./Results/outputGPT3.5_test.txt','r')
lines = f.readlines()
lines = [x for x in lines if x != '\n']

In [4]:
t = ["Thing", "{Label: GearOilTemperature_val, Match: noMatch, Ontology: noMatch file name, Score: 1}"]
for s in t:
    d = re.search('{.*}',s)
    print(d)

None
<re.Match object; span=(0, 86), match='{Label: GearOilTemperature_val, Match: noMatch, O>


In [25]:
pred_labels = []
pred_matches = []
pred_sources = []
for l in lines:
    if re.search('{.*}',l) != None:
        try:
            pred_labels.append(re.search('Label:(.*)Match:', l).group(1)[1:-2].strip().lower())
            pred_matches.append(re.search('Match:(.*)Ontology:', l).group(1)[1:-2].lower())
            pred_sources.append(re.search('Ontology:(.*)Score:', l).group(1)[1:-2].lower())
        except Warning:
            print('Something went wrong')

#pred_matches = [y.replace('<', '').replace('>', '') for y in ['noMatch' if x[0] != '<' else x for x in pred_matches]]

predictions = list(zip(pred_labels,pred_matches,pred_sources))


In [18]:
def evaluate_predictions(predictions, dictionary, path = False):
    tp, fp, tn, fn = 0, 0, 0, 0
    for (l,m,s) in predictions:
        try:
            if not path:
                if (m,s.replace('.txt', '')+'.txt') in dictionary[l]:
                    tp += 1
                elif m == 'nomatch':
                    if (m,m.replace('.txt','')+'.txt') in dictionary[l]:
                        tn += 1
                    elif (m,m.replace('.txt','')+'.txt') not in dictionary[l]:
                        fn += 1
                else:
                    fp += 1
            else:
                for (path,source) in dictionary[l]: 
                    if m in path.split(':') and s.replace('.txt','')+'.txt' == source:
                        tp += 1
                        break
                    elif m == 'nomatch':
                        if m in path.split(':'):
                            tn += 1
                            break
                        elif m not in path.split(':'):
                            fn += 1
                            break
                    else:
                        fp += 1
                        break
        except:
            print(l)
    t = sum([1 for _,x,_ in predictions if x == 'nomatch'])
    print(tp, fp, tn, fn)
    print(f'Labels not found in manual mapping: {len(predictions)-(tp+fp+tn+fn)}')
    print(f'noMatch count: {t}')

    N = len(predictions)
    accuracy = round((tp+tn)/N,3)
    precision = round(tp/(tp+fp),3)
    recall = round(tp/(tp+fn),3)

    print(f'Accuracy: {accuracy}, Precision: {precision}, Recall: {recall}')

In [26]:
evaluate_predictions(predictions, dictionary)
evaluate_predictions(predictions, dictionary2, True)

current inverter set power
59 181 40 279
Labels not found in manual mapping: 1
noMatch count: 373
Accuracy: 0.177, Precision: 0.246, Recall: 0.175
current inverter set power
73 167 40 279
Labels not found in manual mapping: 1
noMatch count: 373
Accuracy: 0.202, Precision: 0.304, Recall: 0.207


In [12]:
for k in dictionary.keys():
    if 'specific' in k.split():
        print(k)

the calculated specific heat loss coefficient ht/a of the building's enclosing structures, w/m2k
specific heat loss coefficient of building,  ht max/a calculated, w/m2k
specific heat loss coefficient of building ventilation hventilation/acalculated, w/m2k


In [17]:
N = 'Nomatch'.lower()
labels = getOntologies()
[x for x,y in labels if y.lower() == N+'.ttl']

[]

In [20]:
[1,2,3,4][:-2]

[1, 2]